Description

Embedding Model : text-embedding-3-small

Vector DB : MongoDB

โดยการทดสอบมีวิธีการที่แตกต่างกัน


Pattern 1

In [81]:
import os
import pandas as pd
import time
import uuid
import nest_asyncio
import asyncio
import numpy as np
from openai import AsyncOpenAI
from pymongo import MongoClient
import fitz  # PyMuPDF
from docx import Document
import tiktoken
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()

nest_asyncio.apply()

In [82]:
#KEY 
OPENAI_API_KEY : str 
EMBEDDING : str 
MONGO_URI : str

MONGO_URI = os.getenv("MONGO_URI")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
EMBEDDING = os.getenv("EMBEDDING")
client_openai = AsyncOpenAI(api_key=OPENAI_API_KEY)
mongo_client = MongoClient(MONGO_URI)
db = mongo_client["vector_db"]
collection = db["vectors"]

In [ ]:
file_path = "../data"
# 3. Define helper functions
def read_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([para.text for para in doc.paragraphs])
    return text

def read_pdf(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

async def embed_batch(batch, embed_model):
    response = await client_openai.embeddings.create(model=embed_model, input=batch)
    return [item.embedding for item in response.data]

async def batch_process_embedding_async(text_list, embed_model, batch_size=100):
    tasks = []
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i + batch_size]
        tasks.append(embed_batch(batch, embed_model))
    results = await asyncio.gather(*tasks)
    embeddings = [embedding for batch in results for embedding in batch]
    return embeddings

def cosine_similarity(vec1, vec2):
    vec1, vec2 = np.array(vec1), np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def count_tokens(text, model="text-embedding-3-small"):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

In [84]:
start_upload = time.perf_counter()

dfs = []

for filename in os.listdir(file_path):
    full_path = os.path.join(file_path, filename)
    if filename.endswith('.csv'):
        df = pd.read_csv(full_path)
        dfs.append(df)
    elif filename.endswith('.xlsx'):
        df = pd.read_excel(full_path, sheet_name=None)
        for sheet in df.values():
            dfs.append(sheet)
    elif filename.endswith('.docx'):
        text = read_docx(full_path)
        df = pd.DataFrame({"text": [text]})
        dfs.append(df)
    elif filename.endswith('.pdf'):
        text = read_pdf(full_path)
        df = pd.DataFrame({"text": [text]})
        dfs.append(df)

df_combined = pd.concat(dfs, ignore_index=True)
df_combined.dropna(inplace=True)
df_combined.drop_duplicates(inplace=True)
df_combined.reset_index(drop=True, inplace=True)

end_upload = time.perf_counter()
print(f"✅ Upload เสร็จ {len(df_combined)} records ในเวลา {end_upload - start_upload:.2f} วินาที")
print("✅ Shape after cleansing:", df_combined.shape)

✅ Upload เสร็จ 250 records ในเวลา 0.01 วินาที
✅ Shape after cleansing: (250, 11)


In [85]:
texts = []
metadata_list = []
for i, row in df_combined.iterrows():
    metadata = row.to_dict()
    text = "\n".join([f"{k}: {v}" for k, v in metadata.items()])
    texts.append(text)
    metadata_list.append((f"vec-{i}", metadata))


In [86]:
start_embed = time.perf_counter()

embeddings = await batch_process_embedding_async(texts, EMBEDDING)

end_embed = time.perf_counter()
embed_time = end_embed - start_embed
print(f"✅ Embedding เสร็จ {len(embeddings)} records ในเวลา {embed_time:.2f} วินาที")

✅ Embedding เสร็จ 250 records ในเวลา 2.87 วินาที


In [91]:
start_upsert = time.perf_counter()

collection.delete_many({})  
documents = []
for (vec_id, metadata), embedding, raw_text in zip(metadata_list, embeddings, texts):
    documents.append({
        "_id": vec_id,
        "embedding": embedding,
        "metadata": metadata,
        "raw_text": raw_text
    })

collection.insert_many(documents)
print(f"✅ Inserted {len(documents)} documents into MongoDB.")

end_upsert = time.perf_counter()
upsert_time = end_upsert - start_upsert
print(f"✅ Upsertเสร็จ {len(documents)} records ในเวลา {upsert_time :.2f} วินาที")


✅ Inserted 250 documents into MongoDB.
✅ Upsertเสร็จ 250 records ในเวลา 0.15 วินาที


In [94]:
async def retrieve_context_from_mongodb(question: str, top_k: int = 50):
    embedder = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)
    question_vector = embedder.embed_query(question)

    documents = list(collection.find())
    similarities = []
    for doc in documents:
        score = cosine_similarity(question_vector, doc["embedding"])
        similarities.append((score, doc))

    similarities.sort(reverse=True, key=lambda x: x[0])
    top_docs = [doc["raw_text"] for score, doc in similarities[:top_k]]
    return "\n".join(top_docs)

In [95]:
start_q = time.perf_counter()

question = "สินค้ามีอะไรบ้าง และมีจำนวนเท่าไร"
context = await retrieve_context_from_mongodb(question)
num_tokens = count_tokens(context)
print(f"จำนวน token ทั้งหมดใน context: {num_tokens}")

จำนวน token ทั้งหมดใน context: 3140


In [96]:
prompt = f"""
ข้อมูลต่อไปนี้มาจากไฟล์ในรูปแบบตาราง เช่น CSV หรือ Excel ซึ่งอาจมีหลายประเภทข้อมูลและหลายแถว:
{context}

คำถามของฉันคือ: "{question}"

กรุณาตอบโดย:
- วิเคราะห์ข้อมูลทั้งหมดให้ครบถ้วน
- สรุปคำตอบโดยใช้ภาษาธรรมดา ชัดเจน และไม่ใช้การจัดรูปแบบตัวหนา หัวข้อ หรือสัญลักษณ์พิเศษ เช่น ** หรือ -
- หากมีหลายรายการสินค้า ให้ระบุชื่อสินค้า พร้อมจำนวน และหน่วยตามลักษณะของสินค้า เช่น เครื่อง, ตัว, คู่, เล่ม ฯลฯ
- หากต้องรวมจำนวน ให้รวมและแสดงยอดรวมทั้งหมด
- ตอบเป็นภาษาไทยแบบเป็นธรรมชาติ เข้าใจง่าย เหมือนอธิบายให้คนทั่วไปฟัง
- หากไม่พบข้อมูลที่เกี่ยวข้อง ให้ตอบว่า "ไม่พบข้อมูลที่เกี่ยวข้อง"
"""

llm = ChatOpenAI(temperature=0, model="gpt-4", api_key=OPENAI_API_KEY)
response = llm.invoke(prompt)

end_q = time.perf_counter()
response_time = end_q - start_q
print("คำตอบ:", response.content)
print(f"⏱ ใช้เวลาในการตอบ: {response_time:.2f} วินาที")

คำตอบ: จากการวิเคราะห์ข้อมูล พบว่ามีสินค้าทั้งหมด 7 ประเภท ดังนี้:

1. T-Shirt จำนวน 3 ตัว
2. Laptop จำนวน 14 เครื่อง
3. Smartphone จำนวน 4 เครื่อง
4. Jeans จำนวน 5 ตัว
5. Refrigerator จำนวน 4 เครื่อง
6. Washing Machine จำนวน 3 เครื่อง
7. Smartwatch จำนวน 5 เครื่อง
8. Running Shoes จำนวน 4 คู่

ดังนั้น สรุปได้ว่ามีสินค้าทั้งหมด 8 ประเภท และมีจำนวนรวมทั้งสิ้น 42 รายการ
⏱ ใช้เวลาในการตอบ: 14.95 วินาที


In [115]:
print(f"✅ Embedding เสร็จทั้งหมด {len(embeddings)} records ในเวลา {embed_time:.2f} วินาที")
print(f"✅ Upsert MongoDB เสร็จ {len(documents)} vectors ในเวลา {upsert_time :.2f} วินาที")
print(f"⏱ ใช้เวลาในการตอบ: {response_time:.2f} วินาที")
print(f"เวลาโดยรวมupload MongoDB ทั้งหมด : {embed_time+upsert_time:.2f} ")

✅ Embedding เสร็จทั้งหมด 250 records ในเวลา 2.87 วินาที
✅ Upsert MongoDB เสร็จ 250 vectors ในเวลา 0.15 วินาที
⏱ ใช้เวลาในการตอบ: 14.95 วินาที
เวลาโดยรวมupload MongoDB ทั้งหมด : 3.01 


Pattern 2

In [113]:
# from transformers import pipeline

# compressor = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

In [114]:
# import sys
# print(sys.executable)